In [1]:
!pip3 install findspark

Defaulting to user installation because normal site-packages is not writeable


In [2]:
!pip3 install numpy

Defaulting to user installation because normal site-packages is not writeable


In [3]:
import findspark
findspark.init('/home/ubuntu/spark-3.3.1-bin-hadoop3')
findspark.find()

'/home/ubuntu/spark-3.3.1-bin-hadoop3'

In [4]:
from pyspark.sql import SparkSession

# The entry point into all functionality in Spark is the SparkSession class.
spark = (SparkSession
	.builder
	.appName("DS5110/CS5501: DDoS detection project")
	.master("spark://172.31.24.89:7077")
	.config("spark.executor.memory", "1024M")
	.getOrCreate())

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/04/19 19:54:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Load csv data into dataframe

In [5]:
df = (spark.read
         .format("csv")
         .option("inferSchema", True)
         .option("header", True)
         .load("hdfs://172.31.24.89:9000/02-21-2018.csv"))

In [6]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator

### Data preprocessing

In [7]:
from pyspark.sql.functions import col, when
#drop unused columns, drop Nan values, convert string labels to integer
df = df.drop('Timestamp')
df = df.na.drop()
df = df.withColumn("Label", when(col("Label") == "Benign", 1).otherwise(0))

In [8]:
#separate features and label columns
features = df.columns[:-1]
label = df.columns[-1]
# Replace missing values in 'Label' with the most common value (mode)
# mode = labels.groupBy("Label").count().orderBy("count", ascending=False).first()["Label"]
# labels = labels.na.fill(value=mode, subset=["Label"])

### Feature selection using PCA

In [9]:
from pyspark.ml.feature import PCA, VectorAssembler

In [12]:
pca_50 = PCA(k=50, inputCol="features", outputCol="pca50Features")  # Reduce to 50 principal components

### Logistic regression 

In [14]:
# Assemble the features into a single vector (necessary for Spark MLlib models)
assembler = VectorAssembler(inputCols=features, outputCol="features")
data = assembler.transform(df)

model = pca_50.fit(data)
data = model.transform(data)

# Select the features vector and the label column
data = data.select("pca50Features", label)
# Split the data into training and test sets
trainData, testData = data.randomSplit([0.7, 0.3])

In [16]:
# Define and fit the Logistic Regression model
import time
start_time = time.time()
lr = LogisticRegression(featuresCol="pca50Features", labelCol=label, maxIter=10, regParam=0.3, elasticNetParam=0.8)
lrModel = lr.fit(trainData)
end_time = time.time()
print(f"Execution time: {end_time - start_time} seconds")

Execution time: 37.73619198799133 seconds


In [17]:
predictions = lrModel.transform(testData)

### Evaluation

In [18]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
# Evaluate the model
evaluator = BinaryClassificationEvaluator(labelCol="Label")
accuracy = evaluator.evaluate(predictions)

precision_evaluator = MulticlassClassificationEvaluator(
    labelCol="Label",
    predictionCol="prediction",
    metricName="precisionByLabel")

# You can specify the label here if you have multiple classes and you are interested in one specific class
precision = precision_evaluator.evaluate(predictions)

# Evaluator for recall
recall_evaluator = MulticlassClassificationEvaluator(
    labelCol="Label",
    predictionCol="prediction",
    metricName="recallByLabel")

# As with precision, specify the label for recall calculation if needed
recall = recall_evaluator.evaluate(predictions)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")


Accuracy: 0.9340538128462116
Precision: 0.6627410031939793
Recall: 0.9981158164308597
